In [ ]:
import json
import networkx as nx
import numpy as np
import itertools

import matplotlib as mpl
import matplotlib.pyplot as plt


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 0.5),
                    textcoords="offset points",
                    ha='center', va='bottom')

with open('data/commonFriends.json', 'r') as file:
    friends_links = json.load(file)

with open('data/myFriends.json', 'r') as file:
    friends_info = json.load(file)
    
friends_info = friends_info['response']['items']

In [ ]:
graph_scale = 0.4

In [ ]:
# graph:
sex = dict(map(lambda d: (d['id'], -d['sex']), friends_info))

G = nx.Graph()
for friend_ID, common_friend_IDs in friends_links.items():
    G.add_edges_from([(int(friend_ID), common_friend_ID) 
                      for common_friend_ID in common_friend_IDs])
nodes_without_links = set(sex)-set(G.nodes)
G.add_nodes_from(nodes_without_links)

# sex

In [ ]:
low, *_, high = sorted(sex.values())
norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
mapper = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.coolwarm)
    
plt.figure(figsize=(20,20))
pos = nx.spring_layout(G, k=graph_scale)
nx.draw(G, pos, nodelist=sex,
        node_color=[mapper.to_rgba(i) 
                    for i in sex.values()],
        width=2, with_labels=False, node_size = 100)
plt.savefig("graphs/friends_sex.png")
plt.show()

# city

In [ ]:
city = dict(map(lambda d: (d['id'], d['city']['id']) 
                if 'city' in d.keys() else (d['id'], -1), friends_info))

In [ ]:
cities_id_map = np.unique(list(city.values()))
cities_id_map = cities_id_map[cities_id_map!=-1]
cities_id_map = {k: v for v, k in enumerate(cities_id_map)}

users_with_city = {k:v for k, v in city.items() if v!=-1}

low, *_, high = sorted(cities_id_map)
norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
mapper_city = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.rainbow)

In [ ]:
G = nx.Graph()
for friend_ID, common_friend_IDs in friends_links.items():
    edges_list = [(int(friend_ID), common_friend_ID) 
                  for common_friend_ID in common_friend_IDs]
    for edge in edges_list:
        if (city[edge[0]]==city[edge[1]])&(city[edge[0]]!=-1):
            color_id = city[edge[0]]
            G.add_edges_from([edge], 
                             color = mapper_city.to_rgba(cities_id_map[color_id]))
        else:
            G.add_edges_from([edge], color = 'white')
            
nodes_without_links = set(sex)-set(G.nodes)
G.add_nodes_from(nodes_without_links)

low, *_, high = sorted(sex.values())
norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
mapper_sex = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.coolwarm)

edges = G.edges()
colors = [G[u][v]['color'] for u,v in edges]

    
plt.figure(figsize=(20,20))
pos = nx.spring_layout(G, k=graph_scale)
  
enone = [(u, v) for (u, v, d) in G.edges(data=True) if d["color"] == "white"]
elook = [(u, v) for (u, v, d) in G.edges(data=True) if d["color"] != "white"]
colors = [G[u][v]['color'] for u,v in elook]


# nodes
nx.draw_networkx_nodes(G, pos, node_size=100, nodelist=sex,
        node_color=[mapper_sex.to_rgba(i) 
                    for i in sex.values()])

# edges
nx.draw_networkx_edges(G, pos, edgelist=enone, alpha=0.0)
nx.draw_networkx_edges(
    G, pos, edgelist=elook, width=2, edge_color=colors, with_labels=False
)

plt.savefig("graphs/friends_city.png")
plt.show()

# personal

## political

In [ ]:
personal = dict(map(lambda d: (d['id'],d['personal']) 
                    if 'personal' in d.keys() else (-1, -1), friends_info))
if -1 in personal:
    del personal[-1]

In [ ]:
political = dict([(k,v['political']) for k, v in personal.items() if 'political' in v.keys()])

In [ ]:
x, height = np.unique(list(political.values()), return_counts=True)

fig, ax = plt.subplots(figsize=(10,5))

rects = ax.bar(x, height)
autolabel(rects)


ax.set_ylabel('Amount')
ax.set_title('My friends\' political views')

ax.legend()

fig.tight_layout()
plt.savefig("graphs/friends_political_bar.png")
plt.show()

In [ ]:
print('Total filling political field: ', np.sum(height))

In [ ]:
low, *_, high = sorted(list(political.values()))
norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
mapper_political = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.spring)

In [ ]:
G = nx.Graph()
for friend_ID, common_friend_IDs in friends_links.items():
    edges_list = [(int(friend_ID), common_friend_ID) 
                  for common_friend_ID in common_friend_IDs]
    if int(friend_ID) in political:
        for edge in edges_list:
            if (edge[1] in political):
                if (political[edge[0]]==political[edge[1]])&(political[edge[0]]!=-1):
                    color_id = political[edge[0]]
                    G.add_edges_from([edge], 
                                     color = mapper_political.to_rgba(color_id))
            else:
                G.add_edges_from([edge], color = 'white', alpha=0.0)
            
nodes_without_links = set(sex)-set(G.nodes)
G.add_nodes_from(nodes_without_links)

low, *_, high = sorted(sex.values())
norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
mapper_sex = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.coolwarm)

edges = G.edges()

plt.figure(figsize=(20,20))
pos = nx.spring_layout(G, k=graph_scale)

enone = [(u, v) for (u, v, d) in G.edges(data=True) if d["color"] == "white"]
elook = [(u, v) for (u, v, d) in G.edges(data=True) if d["color"] != "white"]
colors = [G[u][v]['color'] for u,v in elook]


# nodes
nx.draw_networkx_nodes(G, pos, node_size=100, nodelist=sex,
        node_color=[mapper_sex.to_rgba(i) 
                    for i in sex.values()])

# edges
nx.draw_networkx_edges(G, pos, edgelist=enone, alpha=0.0)
nx.draw_networkx_edges(
    G, pos, edgelist=elook, width=2, edge_color=colors, with_labels=False
)

plt.savefig("graphs/friends_political.png")
plt.show()

## religion

In [ ]:
religion = dict([(k,v['religion']) for k, v in personal.items() if 'religion' in v.keys()])

In [ ]:
x, height = np.unique(list(religion.values()), return_counts=True)

fig, ax = plt.subplots(figsize=(10,5))

rects = ax.bar(x, height)
autolabel(rects)


ax.set_ylabel('Amount')
ax.set_title('My friends\' religion views')

ax.set_xticklabels(x, rotation=90)
ax.legend()

fig.tight_layout()
plt.savefig("graphs/friends_religion_bar.png")
plt.show()

In [ ]:
print('Total filling religion field: ', np.sum(height))

# Schools & Universities

In [ ]:
key_word = 'schools'
key_word = 'universities'

In [ ]:
schools = dict(map(lambda d: (d['id'],d[key_word]) 
                    if key_word in d.keys() else (-1, -1), friends_info))
if -1 in schools:
    del schools[-1]

In [ ]:
prepared_schools_info = {}
for user_id,school_list in schools.items():
    if school_list:
        new_list = [int(school['id']) for school in school_list]
        prepared_schools_info[user_id]=new_list

In [ ]:
unique_schools, height = np.unique(list(itertools.chain(*prepared_schools_info.values())), return_counts=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
rects = ax.bar(range(len(unique_schools)), height)
autolabel(rects)

ax.set_ylabel('Amount')
ax.set_title('My friends\' {} frequencies'.format(key_word))
ax.legend()

fig.tight_layout()
plt.savefig("graphs/friends_{}_bar.png".format(key_word))
plt.show()

In [ ]:
schools_id_map = sorted(np.unique(unique_schools))
schools_id_map = {k: v for v, k in enumerate(schools_id_map)}

low, *_, high = schools_id_map
norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
mapper_schools = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.rainbow)

In [ ]:
G = nx.Graph()
for friend_ID, common_friend_IDs in friends_links.items():
    edges_list = [(int(friend_ID), common_friend_ID) 
                  for common_friend_ID in common_friend_IDs]
    if int(friend_ID) in prepared_schools_info:
        for edge in edges_list:
            if (edge[1] in prepared_schools_info):
                for school_id_left_user in prepared_schools_info[edge[0]]:
                    for school_id_right_user in prepared_schools_info[edge[1]]:
                        if (school_id_left_user==school_id_right_user):
                            color_id = schools_id_map[school_id_left_user]
                            G.add_edges_from([edge], 
                                             color = mapper_schools.to_rgba(color_id))
            else:
                G.add_edges_from([edge], color = 'white')
            
nodes_without_links = set(sex)-set(G.nodes)
G.add_nodes_from(nodes_without_links)

low, *_, high = sorted(sex.values())
norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
mapper_sex = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.coolwarm)

edges = G.edges()
colors = [G[u][v]['color'] for u,v in edges]

    
plt.figure(figsize=(20,20))
pos = nx.spring_layout(G, graph_scale)

enone = [(u, v) for (u, v, d) in G.edges(data=True) if d["color"] == "white"]
elook = [(u, v) for (u, v, d) in G.edges(data=True) if d["color"] != "white"]
colors = [G[u][v]['color'] for u,v in elook]

# nodes
nx.draw_networkx_nodes(G, pos, node_size=100, nodelist=sex,
        node_color=[mapper_sex.to_rgba(i) 
                    for i in sex.values()])

# edges
nx.draw_networkx_edges(G, pos, edgelist=enone, alpha=0.0)
nx.draw_networkx_edges(
    G, pos, edgelist=elook, width=2, edge_color=colors, with_labels=False
)

plt.savefig("graphs/friends_{}.png".format(key_word))
plt.show()

# occupation

In [ ]:
key_word = "type"
#key_word = "name"
info = dict(map(lambda d: (d['id'],d['occupation'][key_word]) 
                    if 'occupation' in d.keys() else (-1, -1), friends_info))
if -1 in info:
    del info[-1]

In [ ]:
x, height = np.unique(list(info.values()), return_counts=True)

fig, ax = plt.subplots(figsize=(10,5))

rects = ax.bar(range(len(x)), height)
autolabel(rects)


ax.set_ylabel('Amount')
ax.set_title('My friends\' occupation')
ax.set_xticks(list(range(len(x))))
ax.set_xticklabels(x, rotation=90)
ax.legend()

fig.tight_layout()
plt.savefig("graphs/friends_occupation_bar.png")
plt.show()

In [ ]:
info_id_map = sorted(np.unique(list(info.values())))
info_id_map = {k: v for v, k in enumerate(info_id_map)}


low, *_, high = list(info_id_map.values())
norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
mapper_info = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.rainbow)

In [ ]:
G = nx.Graph()
for friend_ID, common_friend_IDs in friends_links.items():
    edges_list = [(int(friend_ID), common_friend_ID) 
                  for common_friend_ID in common_friend_IDs]
    if int(friend_ID) in info:
        for edge in edges_list:
            if (edge[1] in info):
                if (info[edge[0]]==info[edge[1]])&(info[edge[0]]!=-1):
                    color_id = info_id_map[info[edge[0]]]
                    G.add_edges_from([edge], 
                                     color = mapper_info.to_rgba(color_id))
            else:
                G.add_edges_from([edge], color = 'white')
            
nodes_without_links = set(sex)-set(G.nodes)
G.add_nodes_from(nodes_without_links)

low, *_, high = sorted(sex.values())
norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
mapper_sex = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.coolwarm)

edges = G.edges()
colors = [G[u][v]['color'] for u,v in edges]

plt.figure(figsize=(20,20))
pos = nx.spring_layout(G, k=graph_scale)


enone = [(u, v) for (u, v, d) in G.edges(data=True) if d["color"] == "white"]
elook = [(u, v) for (u, v, d) in G.edges(data=True) if d["color"] != "white"]
colors = [G[u][v]['color'] for u,v in elook]

# nodes
nx.draw_networkx_nodes(G, pos, node_size=100, nodelist=sex,
        node_color=[mapper_sex.to_rgba(i) 
                    for i in sex.values()])

# edges
nx.draw_networkx_edges(G, pos, edgelist=enone, alpha=0.0)
nx.draw_networkx_edges(
    G, pos, edgelist=elook, width=2, edge_color=colors, with_labels=False
)

plt.savefig("graphs/friends_political.png")
plt.show()